In [1]:
# Automatically reload modules after executing each cell.
%load_ext autoreload
%autoreload 2

In [2]:
# General imports
import numpy as np
import os

# Plotting
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from matplotlib import rc
import matplotlib.font_manager
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

# ML imports
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # pick a number < 4 on ML4HEP; < 3 on Voltan 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2022-03-23 21:27:35.738114: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-03-23 21:28:03.996542: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-03-23 21:28:03.996930: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-03-23 21:28:03.999577: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-03-23 21:28:04.002533: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-03-23 21:28:04.002947: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2

In [4]:
# Data parameters
N = 10**6
mu = 0.1
sigma = 1

In [5]:
### Generate the data.

# Background is Normal(-μ, 1)
# Signal is Normal(μ, 1))
bkgd = np.random.normal(-mu, sigma, N)
sgnl = np.random.normal(mu, sigma, N)
X = np.concatenate([bkgd, sgnl])
y = np.concatenate([np.zeros(N), np.ones(N)])

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
# Loss functions
def bce(y_true, y_pred):
    return -((y_true) * K.log(y_pred + K.epsilon()) + (1. - y_true) * K.log(1. - y_pred))

def mse(y_true, y_pred):
    return -((y_true) * -K.square(1. - y_pred) + (1. - y_true) * -K.square(y_pred + K.epsilon()))

def sqr(y_true, y_pred):
    return -((y_true) * -1. / K.sqrt(y_pred + K.epsilon()) + (1. - y_true) * -K.sqrt(y_pred + K.epsilon()))

def mlc(y_true, y_pred):
    return -((y_true) * K.log(y_pred + K.epsilon()) + (1. - y_true) * (1. - y_pred))

def square_mlc(y_true, y_pred):
    return -((y_true) * K.log( (y_pred + K.epsilon())**2 ) + (1. - y_true) * (1. - y_pred**2))

def exp_mlc(y_true, y_pred):
    return -((y_true) * y_pred + (1. - y_true) * (1. - K.exp(y_pred)))
        
# Likelihood ratios
def lr(x):
    return np.exp(-(1/(2 * sigma**2)) * ( (x - mu)**2 - (x + mu)**2))

def get_bce_lr(model):
    def model_bce_lr(x):
        f = model.predict(x)
        return np.squeeze(f / (1. - f))
    return model_bce_lr

def get_mse_lr(model):
    def model_mse_lr(x):
        f = model.predict(x)
        return np.squeeze(f / (1. - f))
    return model_mse_lr

def get_mlc_lr(model):
    def model_mlc_lr(x):
        f = model.predict(x)
        return np.squeeze(f)
    return model_mlc_lr

def get_square_mlc_lr(model):
    def model_square_mlc_lr(x):
        f = model.predict(x)
        return np.squeeze(f**2)
    return model_square_mlc_lr

def get_exp_mlc_lr(model):
    def model_exp_mlc_lr(x):
        f = model.predict(x)
        return np.squeeze(np.exp(f))
    return model_exp_mlc_lr

def get_sqr_lr(model):
    def model_sqr_lr(x):
        f = model.predict(x)
        return np.squeeze(f)
    return model_sqr_lr

In [7]:
earlystopping = EarlyStopping(patience=10,
                              verbose=0,
                              restore_best_weights=True)

In [8]:
# Models
def train(loss, 
          hidden='relu', 
          output='sigmoid', 
          dropout=True, 
          optimizer='adam', 
          metrics=['accuracy'], 
          verbose=0):
    model = Sequential()
    if dropout:
        model.add(Dense(64, activation=hidden, input_shape=(1, )))
        model.add(Dropout(0.1))
        model.add(Dense(128, activation=hidden))
        model.add(Dropout(0.1))
        model.add(Dense(64, activation=hidden))
        model.add(Dropout(0.1))
        model.add(Dense(1, activation=output))
    else: 
        model.add(Dense(64, activation=hidden, input_shape=(1, )))
        model.add(Dense(128, activation=hidden))
        model.add(Dense(64, activation=hidden))
        model.add(Dense(1, activation=output))        
    
    model.compile(loss=loss,
                  optimizer=optimizer, 
                  metrics=metrics)
    
    trace = model.fit(X_train, 
                      y_train,
                      epochs = 100, 
                      batch_size=int(0.1*N), 
                      validation_data=(X_test, y_test),
                      callbacks=[earlystopping], 
                      verbose=verbose)
    print(trace.history['val_loss'][-1], end = ' ')
    
    lr_match = {bce: get_bce_lr, 
                mse: get_mse_lr, 
                mlc: get_mlc_lr,
                square_mlc: get_square_mlc_lr,
                exp_mlc: get_exp_mlc_lr,
                sqr: get_sqr_lr}
    model_lr = lr_match[loss](model)
    
    return model, model_lr

In [9]:
# Plotting functions
def get_preds(model_lrs, xs=np.linspace(-6, 6, 1000)):
    # Takes in model_lrs, a list of model likelihood ratios and xs, a list of 
    # values on which to compute the likelihood ratios. Returns a 2D array. The 
    # nth row is the likelihood ratio predictions from the nth model in 
    # model_lrs.
    return np.array([model_lr(xs) for model_lr in model_lrs])
    
def avg_lr(preds):
    # Takes in a 2D array of multiple models' likelihood ratio predictions. 
    # Returns the average likelihood ratio prediction and its error.
    return preds.mean(axis=0), preds.std(axis=0)

def avg_lrr(preds, xs=np.linspace(-6, 6, 1000)):
    # Takes in a 2D array of multiple models' likelihood ratio predictions. 
    # Returns the average ratio of predicted likelihood to true likelihood and 
    # its error.
    lrr_preds = preds / lr(xs)
    return lrr_preds.mean(axis=0), lrr_preds.std(axis=0)
    
def lr_plot(ensembles,
            filename=None,
            cs = ['brown', 'green', 'red', 'blue'],
            lss = [':', '--', '-.', ':'],
            xs=np.linspace(-6, 6, 1000)):
    # Takes in a list of pairs (lr_avg, lr_err). Plots them against the true 
    # likelihood.
    fig, ax_1 = plt.subplots(figsize = (10, 6))
    
    # Plot true likelihood
    plt.plot(xs, lr(xs), label = 'Exact', c='k', ls='-')
    
    # Plot model likelihoods
    for i in range(len(ensembles)):
        avg, err, lbl = ensembles[i]
        plt.plot(xs, avg, label=lbl, c=cs[i], ls=lss[i])
        plt.fill_between(xs, avg - err, avg + err, color=cs[i], alpha=0.1)
    plt.legend()
    ax_1.minorticks_on()
    ax_1.tick_params(direction='in', which='both',length=5)
    plt.ylabel('Likelihood Ratio')

    # Plot background and signal
    ax_2 = ax_1.twinx()
    bins = np.linspace(-6, 6, 100)
    plt.hist(sgnl, alpha=0.1, bins=bins)
    plt.hist(bkgd, alpha=0.1, bins=bins)
    ax_2.minorticks_on()
    ax_2.tick_params(direction='in', which='both',length=5)
    plt.ylabel('Count')

    plt.xlim(-6, 6)
    plt.xlabel(r'$x$')
    plt.title(r"$\mu_{\rm{sgnl}}="+str(mu)+r", \mu_{\rm{bkgd}}="+str(-mu)+r"$",loc="right",fontsize=20);
    if filename != None:
        plt.savefig(filename, dpi=1200, bbox_inches='tight')

def lrr_plot(ensembles,
             filename=None,
             cs = ['brown', 'green', 'red', 'blue'],
             lss = [':', '--', '-.', ':'],
             xs=np.linspace(-6, 6, 1000)):
    # Takes in a list of pairs (lrr_avg, lrr_err). Plots them.
    fig, ax_1 = plt.subplots(figsize = (10, 6))
    
    # Plot ratios of likelihood ratios
    for i in range(len(ensembles)):
        avg, err, lbl = ensembles[i]
        plt.plot(xs, avg, label=lbl, c=cs[i], ls=lss[i])
        plt.fill_between(xs, avg - err, avg + err, color=cs[i], alpha=0.1)
    plt.axhline(1,ls=":",color="grey", lw=0.5)
    plt.axvline(0,ls=":",color="grey", lw=0.5)
    plt.legend()
    ax_1.minorticks_on()
    ax_1.tick_params(direction='in', which='both',length=5)
    plt.ylim(0.94, 1.06)
    plt.ylabel('Ratio')

    # Plot background and signal
    ax_2 = ax_1.twinx()
    bins = np.linspace(-6, 6, 100)
    plt.hist(sgnl, alpha=0.1, bins=bins)
    plt.hist(bkgd, alpha=0.1, bins=bins)
    ax_2.minorticks_on()
    ax_2.tick_params(direction='in', which='both',length=5)
    plt.ylabel('Count')

    plt.xlim(-6, 6)
    plt.xlabel(r'$x$')
    plt.title(r"$\mu_{\rm{sgnl}}="+str(mu)+r", \mu_{\rm{bkgd}}="+str(-mu)+r"$",loc="right",fontsize=20)
    if filename != None:
        plt.savefig(filename, dpi=1200, bbox_inches='tight')
    
#def mae_plot(model_lrs):

# Reweighting Schemes

In [ ]:
reps = 100

# model parameters
params_1 = {'loss':mlc, 'hidden':'relu', 'output':'relu'}
params_2 = {'loss':square_mlc, 'hidden':'relu', 'output':'linear'}
params_3 = {'loss':exp_mlc, 'hidden':'relu', 'output':'linear'}

In [ ]:
%%time

models_1 = [None] * reps
models_2 = [None] * reps
models_3 = [None] * reps

lrs_1 = [None] * reps
lrs_2 = [None] * reps
lrs_3 = [None] * reps

for i in range(reps):
    print(i, end = ' ')
    models_1[i], lrs_1[i] = train(**params_1)
    models_2[i], lrs_2[i] = train(**params_2)
    models_3[i], lrs_3[i] = train(**params_3)
    print()

In [ ]:
for i in range(reps):
    models_1[i].save_weights('models/ratios/mlc/mlc_model_{}.h5'.format(i))
    models_2[i].save_weights('models/ratios/mlc/square_mlc_model_{}.h5'.format(i))
    models_3[i].save_weights('models/ratios/mlc/exp_mlc_model_{}.h5'.format(i))

In [ ]:
# Get average predictions and errors. Add on the labels to 
# make plotting easier.
lr_1 = avg_lr(get_preds(lrs_1)) + ('linear',)
lr_2 = avg_lr(get_preds(lrs_2)) + ('square',)
lr_3 = avg_lr(get_preds(lrs_3)) + ('exponential',)

lrr_1 = avg_lrr(get_preds(lrs_1)) + ('linear',)
lrr_2 = avg_lrr(get_preds(lrs_2)) + ('square',)
lrr_3 = avg_lrr(get_preds(lrs_3)) + ('exponential',)

In [ ]:
lr_plot([lr_1, lr_2, lr_3], 'plots/mlc_param_lr.png')

In [ ]:
lrr_plot([lrr_1, lrr_2, lrr_3], 'plots/mlc_param_lrrs.png')

In [ ]:
1 / 0

# Error Calculations

In [10]:
reps = 100

bce_params = {'loss':bce}
mse_params = {'loss':mse}
mlc_params = {'loss':exp_mlc, 'hidden':'relu', 'output':'linear'}
sqr_params = {'loss':sqr, 'output':'relu'}

Ns = 10**np.arange(2, 9)

In [11]:
def mae(model_lr):
    # Takes in model_lr, a model likelihood ratio. Returns the expected absolute
    # error for that model.
    return np.abs(model_lr(X) - lr(X)).mean()

In [ ]:
%%time

bce_models = {}
bce_lrs = {}
for N in Ns:
    print('===================================================\n{}'.format(N))
    bce_models[N] = [None] * reps
    bce_lrs[N] = [None] * reps
    
    # Generate data
    bkgd = np.random.normal(-mu, 1, N)
    sgnl = np.random.normal(mu, 1, N)
    X = np.concatenate([bkgd, sgnl])
    y = np.concatenate([np.zeros(N), np.ones(N)])

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    for i in range(reps):
        print(i, end = ' ')
        bce_models[N][i], bce_lrs[N][i] = train(**bce_params)
        print()

100
0 

2022-03-23 21:28:04.523623: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-03-23 21:28:04.530694: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2994530000 Hz
2022-03-23 21:28:04.531874: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e7adb1c4e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-03-23 21:28:04.531900: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-03-23 21:28:04.640794: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e7adb863d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-03-23 21:28:04.640824: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 6000, Compute Capability 7.5
2022-03-23 21:28:04.642168: I 

0.6990693807601929 
1 0.6992847919464111 
2 0.6991275548934937 
3 0.6978055834770203 
4 0.7003640532493591 
5 0.7020862102508545 
6 0.7002569437026978 
7 0.7039097547531128 
8 0.6994238495826721 
9 0.7010352611541748 
10 0.7036917209625244 
11 0.6981343030929565 
12 0.7014122009277344 
13 0.6966906189918518 
14 0.697354793548584 
15 0.7018502950668335 
16 0.6980582475662231 
17 0.7014383673667908 
18 0.6979420185089111 
19 

In [ ]:
# Calculate mean absolute errors.

N = 10**4

bkgd = np.random.normal(-mu, 1, N)
sgnl = np.random.normal(mu, 1, N)
X = np.concatenate([bkgd, sgnl])

bce_mae_avg = []
bce_mae_err = []
for N in Ns:
    bce_maes = [mae(lr) for lr in bce_lrs[N]]
    
    bce_mae_avg += [np.mean(bce_maes)]
    bce_mae_err += [np.std(bce_maes)]

bce_mae_avg = np.array(bce_mae_avg)
bce_mae_err = np.array(bce_mae_err)

In [ ]:
for N in Ns:
    for i in range(reps):
        bce_models[N][i].save_weights('models/maes/bce/bce_model_{}_{}.h5'.format(N, i))

np.save('lr_data/bce_mae_avg.npy', bce_mae_avg)
np.save('lr_data/bce_mae_err.npy', bce_mae_err)

In [ ]:
%%time

bce_models = {}
mse_models = {}
mlc_models = {}
sqr_models = {}

bce_lrs = {}
mse_lrs = {}
mlc_lrs = {}
sqr_lrs = {}

for N in Ns:
    print('===================================================\n{}'.format(N))
    # Set up lists
    bce_models[N] = [None] * reps
    mse_models[N] = [None] * reps
    mlc_models[N] = [None] * reps
    sqr_models[N] = [None] * reps
    
    bce_lrs[N] = [None] * reps
    mse_lrs[N] = [None] * reps
    mlc_lrs[N] = [None] * reps
    sqr_lrs[N] = [None] * reps
    
    # Generate data
    bkgd = np.random.normal(-mu, 1, N)
    sgnl = np.random.normal(mu, 1, N)
    X = np.concatenate([bkgd, sgnl])
    y = np.concatenate([np.zeros(N), np.ones(N)])

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    for i in range(reps):
        print(i, end = ' ')
        bce_lrs[N][i] = train(**bce_params)[1]
        mse_lrs[N][i] = train(**mse_params)[1]
        mlc_lrs[N][i] = train(**mlc_params)[1]
        sqr_lrs[N][i] = train(**sqr_params)[1]
        print()

In [ ]:
# Calculate mean absolute errors.

N = 10**3

bkgd = np.random.normal(-mu, 1, N)
sgnl = np.random.normal(mu, 1, N)
X = np.concatenate([bkgd, sgnl])

bce_mae_avg = []
mse_mae_avg = []
mlc_mae_avg = []
sqr_mae_avg = []

bce_mae_err = []
mse_mae_err = []
mlc_mae_err = []
sqr_mae_err = []

for N in Ns:
    bce_maes = [mae(lr) for lr in bce_lrs[N]]
    mse_maes = [mae(lr) for lr in mse_lrs[N]]
    mlc_maes = [mae(lr) for lr in mlc_lrs[N]]
    sqr_maes = [mae(lr) for lr in sqr_lrs[N]]
    
    bce_mae_avg += [np.mean(bce_maes)]
    bce_mae_err += [np.std(bce_maes)]
    mse_mae_avg += [np.mean(mse_maes)]
    mse_mae_err += [np.std(mse_maes)]
    
    mlc_mae_avg += [np.mean(mlc_maes)]
    mlc_mae_err += [np.std(mlc_maes)]
    
    sqr_mae_avg += [np.mean(sqr_maes)]
    sqr_mae_err += [np.std(sqr_maes)]

bce_mae_avg = np.array(bce_mae_avg)
mse_mae_avg = np.array(mse_mae_avg)
mlc_mae_avg = np.array(mlc_mae_avg)
sqr_mae_avg = np.array(sqr_mae_avg)

bce_mae_err = np.array(bce_mae_err)
mse_mae_err = np.array(mse_mae_err)
mlc_mae_err = np.array(mlc_mae_err)
sqr_mae_err = np.array(sqr_mae_err)

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

plt.plot(Ns, bce_mae_avg, c='brown', ls=':', label='BCE')
plt.plot(Ns, mse_mae_avg, c='green', ls='--', label='MSE')
plt.plot(Ns, mlc_mae_avg, c='red', ls='--', label='MLC')
plt.plot(Ns, sqr_mae_avg, c='blue', ls='-.', label='SQR')
plt.fill_between(Ns, bce_mae_avg - bce_mae_err, bce_mae_avg + bce_mae_err, color='brown', alpha=0.1)
plt.fill_between(Ns, mse_mae_avg - mse_mae_err, mse_mae_avg + mse_mae_err, color='green', alpha=0.1)
plt.fill_between(Ns, mlc_mae_avg - mlc_mae_err, mlc_mae_avg + mlc_mae_err, color='red', alpha=0.1)
plt.fill_between(Ns, sqr_mae_avg - sqr_mae_err, sqr_mae_avg + sqr_mae_err, color='blue', alpha=0.1)
plt.legend()

plt.xscale("log", base=10)
plt.minorticks_on()
plt.tick_params(direction='in', which='both',length=5)
plt.ylabel('Mean Absolute Error')
plt.xlabel(r'$N$')

plt.title(r"$\mu_{\rm{sgnl}}="+str(mu)+r", \mu_{\rm{bkgd}}="+str(-mu)+r"$",loc="right",fontsize=20);
plt.savefig('plots/maes.png', dpi=1200, bbox_inches='tight')

# Neural Network Training

In [ ]:
reps = 10

# Parameters to compare
params_1 = {'loss':sqr, 'hidden':'relu', 'output':'relu'}
params_2 = {'loss':sqr, 'hidden':'relu', 'output':'elu'}
params_3 = {'loss':sqr, 'hidden':'elu', 'output':'relu'}
params_4 = {'loss':sqr, 'hidden':'elu', 'output':'elu'}

In [ ]:
%%time

models_1 = [None] * reps
models_2 = [None] * reps
models_3 = [None] * reps
models_4 = [None] * reps

for i in range(reps):
    print(i, end = ' ')
    models_1[i] = train(**params_1)[1]
    models_2[i] = train(**params_2)[1]
    models_3[i] = train(**params_3)[1]
    models_4[i] = train(**params_4)[1]
    print()

In [ ]:
# Get average predictions and errors. Add on the labels to 
# make plotting easier.
sqr_lr_1 = avg_lr(get_preds(models_1)) + ('ReLU/ReLU',)
sqr_lr_2 = avg_lr(get_preds(models_2)) + ('ReLU/ELU',)
sqr_lr_3 = avg_lr(get_preds(models_3)) + ('ELU/ReLU',)
sqr_lr_4 = avg_lr(get_preds(models_4)) + ('ELU/ELU',)

sqr_lrr_1 = avg_lrr(get_preds(models_1)) + ('ReLU/ReLU',)
sqr_lrr_2 = avg_lrr(get_preds(models_2)) + ('ReLU/ELU',)
sqr_lrr_3 = avg_lrr(get_preds(models_3)) + ('ELU/ReLU',)
sqr_lrr_4 = avg_lrr(get_preds(models_4)) + ('ELU/ELU',)

In [ ]:
lr_plot([sqr_lr_1, sqr_lr_2, sqr_lr_3, sqr_lr_4], 'plots/sqr_lr_comparisons.png')

In [ ]:
lrr_plot([sqr_lrr_1, sqr_lrr_2, sqr_lrr_3, sqr_lrr_4], 'plots/sqr_lrr_comparisons.png')

In [ ]:
xs = np.linspace(-6, 6, 1000)

sqr_lrs_1 = np.array([sqr_lr(xs) for sqr_lr in models_1[100:200]])
sqr_lrs_2 = np.array([sqr_lr(xs) for sqr_lr in models_2[100:200]])
sqr_lrs_3 = np.array([sqr_lr(xs) for sqr_lr in models_3[100:200]])
sqr_lrs_4 = np.array([sqr_lr(xs) for sqr_lr in models_4[100:200]])

lrr_1 = sqr_lrs_1 / lr(xs)
lrr_2 = sqr_lrs_2 / lr(xs)
lrr_3 = sqr_lrs_3 / lr(xs)
lrr_4 = sqr_lrs_4 / lr(xs)

lrr_bar_1 = lrr_1.mean(axis=0)
lrr_bar_2 = lrr_2.mean(axis=0)
lrr_bar_3 = lrr_3.mean(axis=0)
lrr_bar_4 = lrr_4.mean(axis=0)

lrr_err_1 = lrr_1.std(axis=0)
lrr_err_2 = lrr_2.std(axis=0)
lrr_err_3 = lrr_3.std(axis=0)
lrr_err_4 = lrr_4.std(axis=0)

In [ ]:
plt.plot(xs, lrr_bar_1, label = 'relu/relu', c = 'brown', ls=':')
plt.plot(xs, lrr_bar_2, label = 'relu/elu', c = 'green', ls='--')
plt.plot(xs, lrr_bar_3, label = 'elu/relu', c = 'blue', ls='-.')
plt.plot(xs, lrr_bar_4, label = 'elu/elu', c = 'red', ls='--')
plt.fill_between(xs, lrr_bar_1 - lrr_err_1, lrr_bar_1 + lrr_err_1, color='brown', alpha=0.1)
plt.fill_between(xs, lrr_bar_2 - lrr_err_2, lrr_bar_2 + lrr_err_2, color='green', alpha=0.1)
plt.fill_between(xs, lrr_bar_3 - lrr_err_3, lrr_bar_3 + lrr_err_3, color='blue', alpha=0.1)
plt.fill_between(xs, lrr_bar_4 - lrr_err_4, lrr_bar_4 + lrr_err_4, color='red', alpha=0.1)
plt.axhline(1,ls=":",color="grey", lw=0.5)
plt.axvline(0,ls=":",color="grey", lw=0.5)
plt.legend()
plt.savefig('plots/sqr_cmp.png')

In [ ]:
def mae(data):
    return np.abs(data - lr(xs)).sum(axis=1).mean(axis=0)

mae(sqr_lrs_1), mae(sqr_lrs_2), mae(sqr_lrs_3), mae(sqr_lrs_4)

In [ ]:
1/0

In [ ]:
plt.plot(xs, lr(xs), c='k', label='exact')
plt.plot(xs, bce_rs_mean, c='green', label='relu/relu')
plt.plot(xs, bce_es_mean, c='red', label='elu/elu')

plt.fill_between(xs, bce_rs_mean - bce_rs_err, bce_rs_mean + bce_rs_err, color='green', alpha=0.1)
plt.fill_between(xs, bce_es_mean - bce_es_err, bce_es_mean + bce_es_err, color='red', alpha=0.1)
plt.legend();
plt.ylim(0, 4)
plt.savefig('plots/avg_lr_mlc_ee_rr_2.png', dpi=1200, bbox_inches='tight')

In [ ]:
plt.plot(xs, bce_rs_mean / lr(xs), c='green', label='relu/relu')
plt.plot(xs, bce_es_mean / lr(xs), c='red', label='elu/elu')
plt.fill_between(xs, bce_rsr_mean - bce_rsr_err, bce_rsr_mean + bce_rsr_err, color='green', alpha=0.1)
plt.fill_between(xs, bce_esr_mean - bce_esr_err, bce_esr_mean + bce_esr_err, color='red', alpha=0.1)

#plt.ylim(0.5, 1.5)
plt.legend();
plt.savefig('plots/avg_lrr_mlc_eerr_1.png', dpi=1200, bbox_inches='tight')

In [ ]:
mlc_rs_maes = []
mlc_es_maes = []

# This should be a sum
for i in range(100):
    model_mlc_lr = get_mlc_lr(bce_rs_models[i])
    mlc_rs_maes += [np.mean(np.abs(model_mlc_lr(X) - lr(X)))]
    
    model_mlc_lr = get_mlc_lr(bce_es_models[i])
    mlc_es_maes += [np.mean(np.abs(model_mlc_lr(X) - lr(X)))]

In [ ]:
"""
22 mins sqr rs/es
20 mins sqr rr/ee
30 mins mlc rr/ee dropout
"""

# Ensemble Learning

In [ ]:
1/0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras import Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Lambda, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.models import Model
    
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from matplotlib import gridspec
from matplotlib import patches

plt.rcParams["font.family"] = "serif"

In [ ]:
#MLC loss

N = 100000

theta0 = 0.1

X_MC = np.random.normal(-theta0,1,N)
X_data = np.random.normal(theta0,1,N)
X_MLE_train, X_MLE_val, Y_MLE_train, Y_MLE_val = train_test_split(np.concatenate([X_data,X_MC]), np.concatenate([np.ones(len(X_data)),np.zeros(len(X_MC))]), test_size=0.5)


def CustomLoss2(y_true, y_pred):
    return -y_true*K.log(y_pred**2+0.00000001) + (1.-y_true)*y_pred**2

def CustomLoss(y_true, y_pred):
    return -y_true*K.log(y_pred+0.00000001) + (1.-y_true)*y_pred

model_MLE2 = Sequential()
model_MLE2.add(Dense(64, activation='relu',input_shape =(1,))) 
model_MLE2.add(Dense(128, activation='relu'))
model_MLE2.add(Dense(64, activation='relu'))
model_MLE2.add(Dense(1, activation='linear')) #was sigmoid
model_MLE2.compile(loss=lambda y_true, y_pred: CustomLoss2(y_true, y_pred), optimizer='adam', metrics=['accuracy'])
hist_MLE2 = model_MLE2.fit(X_MLE_train, Y_MLE_train, epochs=100, batch_size=int(0.01*len(X_MLE_train)),validation_data=(X_MLE_val, Y_MLE_val),callbacks=[earlystopping])

model_MLE = Sequential()
model_MLE.add(Dense(64, activation='relu',input_shape =(1,))) 
model_MLE.add(Dense(128, activation='relu'))
model_MLE.add(Dense(64, activation='relu'))
model_MLE.add(Dense(1, activation='relu')) #was sigmoid
model_MLE.compile(loss=lambda y_true, y_pred: CustomLoss(y_true, y_pred), optimizer='adam', metrics=['accuracy'])
hist_MLE = model_MLE.fit(X_MLE_train, Y_MLE_train, epochs=100, batch_size=int(0.01*len(X_MLE_train)),validation_data=(X_MLE_val, Y_MLE_val),callbacks=[earlystopping])

model_BCE = Sequential()
model_BCE.add(Dense(64, activation='relu',input_shape =(1,))) 
model_BCE.add(Dense(128, activation='relu'))
model_BCE.add(Dense(64, activation='relu'))
model_BCE.add(Dense(1, activation='sigmoid'))
model_BCE.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
hist_BCE = model_BCE.fit(X_MLE_train, Y_MLE_train, epochs=100, batch_size=int(0.01*len(X_MLE_train)),validation_data=(X_MLE_val, Y_MLE_val),callbacks=[earlystopping])


In [ ]:
fig = plt.figure(figsize=(8, 6))
gs = gridspec.GridSpec(1, 1) 
ax1 = plt.subplot(gs[0])
ax1.yaxis.set_ticks_position('both')
ax1.xaxis.set_ticks_position('both')
ax1.tick_params(direction="in",which="both")
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax1.minorticks_on()

xx = np.linspace(-3,3,100)
preds_BCE = model_BCE.predict(xx)
plotBCE = plt.plot(xx,preds_BCE/(1.-preds_BCE),label="NN (BCE)",ls=":",lw=5,color='green')
preds_test2 = model_MLE2.predict(xx)**2
plot12 = plt.plot(xx,preds_test2,label="NN (MLC, squared)",ls=":",color='green')
preds_test = model_MLE.predict(xx)
plot11 = plt.plot(xx,preds_test,label="NN (MLC, linear)",ls="-",color='green')
plot2 = plt.plot(xx,np.exp(-((xx-theta0)**2-(xx+theta0)**2)/(2*1**2)),label="exact",color='orange')
plt.ylim([0,2])
plt.axvline(0,ls=":",color="grey")
plt.axhline(1,ls=":",color="grey")
plt.xlabel(r"$x$",fontsize=20)
plt.ylabel("likelihood ratio",fontsize=20)
plt.title(r"$\mu_{MC} = "+str(theta0)+r", \mu_{data}=0.5$",loc="right",fontsize=20)

ax2 = ax1.twinx()
_,_,_=plt.hist(X_MLE_train[Y_MLE_train==0],bins=np.linspace(-3,3,50),alpha=0.3,label="Back.",color='red')
_,_,_=plt.hist(X_MLE_train[Y_MLE_train==1],bins=np.linspace(-3,3,50),alpha=0.3,label="Signal")
ax2.tick_params(axis='y', labelcolor="red")
ax2.set_ylabel("histogram",color='red',fontsize=20)

for label in ax2.yaxis.get_majorticklabels():
        label.set_fontsize(20)

leg = plt.legend([plotBCE[0], plot11[0],plot12[0],plot2[0]],['BCE', 'MLC, linear','MLC, squared','Exact'], loc="upper left",frameon=False,fontsize=18)
plt.legend(frameon=False,fontsize=20)
plt.gca().add_artist(leg)

#plt.savefig("ensembleLearnPlots/MLCloss.pdf",bbox_inches='tight')